In [1]:
"#@title" 
import numpy as np
# import pydicom as pd
# import dicompylercore as dc
from shapely.geometry import Polygon, Point
# from dicompylercore import dicomparser

from __future__ import print_function

import os 
from os.path import isdir, isfile
import glob
# import png
from PIL import Image


In [2]:
import pickle
with (open('allpatientcurves.pkl', "rb")) as openfile:
    patient_seg_dict = pickle.load(openfile)


In [3]:
# import pickle
with (open('patientcurvatures.pkl', "rb")) as openfile:
    patient_seg_cur_dict = pickle.load(openfile)


In [4]:
import os 
from os.path import isdir, isfile
import glob
import png
from PIL import Image
# !pip install pypng

In [22]:
n = 12
gap = 24

'''Creating static graph structure for all instances'''
patient_seg_adj_dict = {}
for i in range(n + gap):
    patient_seg_adj_dict[i] = []

for i in range(n):
    for j in range(n):
        if i!=j:
            patient_seg_adj_dict[i].append(j)

add_co = n
for i in range(n):
    patient_seg_adj_dict[i].append(add_co)
    patient_seg_adj_dict[add_co].append(i)
    patient_seg_adj_dict[add_co].append(add_co + 1)
    patient_seg_adj_dict[add_co + 1].append(add_co)
    add_co+=1
    if i+1 != n:
        patient_seg_adj_dict[i+1].append(add_co)
        patient_seg_adj_dict[add_co].append(i+1)
        add_co+=1
    else:
        patient_seg_adj_dict[0].append(add_co)
        patient_seg_adj_dict[add_co].append(0)

'''Calculating nodes coordinates for the graphs'''
patient_seg_nodes_dict = {}
for patient in patient_seg_dict.keys():
    patient_seg_nodes_dict[patient] = {}
    for imageposno in patient_seg_dict[patient].keys():
        patient_seg_nodes_dict[patient][imageposno] = [{},{},{},{},{},{}]
        curve = patient_seg_dict[patient][imageposno][0]
        curve_curvs = patient_seg_cur_dict[patient][imageposno][0]
        length_seg = len(curve) / n
        length_gap = len(curve) / gap
        max_curv_index_first = 0
        prev_max_curv_index = - length_gap
        add_co = n
        counter = -1
        for idx0, idx1 in [[length_seg*i, length_seg*(i+1)] for i in range(n)]:
            counter += 1
            if idx1 != length_seg*n:
                idx0 = max([idx0 , prev_max_curv_index + length_gap])
                max_curv_index = curve_curvs.index(max(curve_curvs[idx0 : idx1]))
                if not (max_curv_index >= idx0 and max_curv_index < idx1):
                     max_curv_index = (idx0 + idx1) / 2
                if counter == 0:
                    max_curv_index_first = max_curv_index
                else:
                    index1 = prev_max_curv_index + (max_curv_index - prev_max_curv_index)/3
                    index2 = prev_max_curv_index + 2*(max_curv_index - prev_max_curv_index)/3
                    patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index1]
                    add_co+=1
                    patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index2]    
                    add_co+=1
                patient_seg_nodes_dict[patient][imageposno][0][counter] = curve[max_curv_index]
                prev_max_curv_index = max_curv_index
            else:
                idx0 = max([idx0 , prev_max_curv_index + length_gap])
                idx1 = min([len(curve) , len(curve) + (max_curv_index_first - length_gap)])
                max_curv_index = curve_curvs.index(max(curve_curvs[idx0 : idx1]))
                if not (max_curv_index >= idx0 and max_curv_index < idx1):
                     max_curv_index = (idx0 + idx1) / 2
                index1 = prev_max_curv_index + (max_curv_index - prev_max_curv_index)/3
                index2 = prev_max_curv_index + 2*(max_curv_index - prev_max_curv_index)/3
                patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index1]
                add_co+=1
                patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index2]    
                add_co+=1
                patient_seg_nodes_dict[patient][imageposno][0][counter] = curve[max_curv_index]

                last_gap = len(curve) + max_curv_index_first - max_curv_index
                index1 = max_curv_index + last_gap/3
                index2 = max_curv_index + 2*last_gap/3
                if index1 >= len(curve):
                    index1 = index1 - len(curve)
                if index2 >= len(curve):
                    index2 = index2 - len(curve)
                patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index1]
                add_co+=1
                patient_seg_nodes_dict[patient][imageposno][0][add_co] = curve[index2] 
            # print(patient_seg_nodes_dict, max_curv_index, prev_max_curv_index, index1, index2)
# print(patient_seg_adj_dict)

In [23]:
from scipy.spatial import distance
max_dst = 0
patient_seg_features_dict = {}
for patient in patient_seg_nodes_dict.keys():
    patient_seg_features_dict[patient] = {}
    for imageposno in patient_seg_nodes_dict[patient].keys():
        patient_seg_features_dict[patient][imageposno] = [{},{},{},{},{},{}]
        for i in range(36):
            patient_seg_features_dict[patient][imageposno][0][i] = []
            a = patient_seg_nodes_dict[patient][imageposno][0][i]
            for j in range(36):
                if i == j:
                    patient_seg_features_dict[patient][imageposno][0][i].append(0.0)
                    continue
                b = patient_seg_nodes_dict[patient][imageposno][0][j]
                dst = distance.euclidean(a, b)
                patient_seg_features_dict[patient][imageposno][0][i].append(dst)
                if dst > max_dst:
                    max_dst = dst
max_dst


453.3188723183715

In [33]:
from scipy.spatial import distance
max_dst = 0
min_dst = 0
patient_seg_features2_dict = {}
for patient in patient_seg_nodes_dict.keys():
    patient_seg_features2_dict[patient] = {}
    for imageposno in patient_seg_nodes_dict[patient].keys():
        patient_seg_features2_dict[patient][imageposno] = [{},{},{},{},{},{}]
        for i in range(36):
            patient_seg_features2_dict[patient][imageposno][0][i] = []
            a = patient_seg_nodes_dict[patient][imageposno][0][i]
            for j in range(36):
                if i == j:
                    patient_seg_features2_dict[patient][imageposno][0][i].append(0.0)
                    continue
                b = patient_seg_nodes_dict[patient][imageposno][0][j]
                if distance.euclidean(a, b) == 0.0:
                    patient_seg_features2_dict[patient][imageposno][0][i].append(0.0)
                    continue
                dst = (float(b[0] - a[0]))/distance.euclidean(a, b)
                patient_seg_features2_dict[patient][imageposno][0][i].append(dst)
                if dst > max_dst:
                    max_dst = dst
                if dst < min_dst:
                    min_dst = dst
max_dst, min_dst


(1.0, -1.0)

In [34]:
print((patient_seg_features2_dict['S205']['-45.0.dcm.png.png'][0]))

{0: [0.0, -0.21693045781865616, -0.006289183791748741, 0.23372653240486918, 0.5271366041964244, 0.8272554853422766, 0.9177221168073145, 0.9917374158411135, 0.8382376284987111, 0.46441051437306946, 0.2580043016648466, -0.04440061362072039, -0.04163054471218133, -0.18428853505018536, -0.15646472049533244, -0.06906545518773156, 0.06099731794136101, 0.14420545377163382, 0.334276824516433, 0.4415676795001447, 0.6397315948509501, 0.7401442508235887, 0.8610153295679583, 0.8907990361361506, 0.9704258344078877, 0.9982226157912, 0.9620639325477691, 0.9136230744341561, 0.7233555441435721, 0.5905099943842171, 0.4025538059337346, 0.3276732207983762, 0.1559625734730109, 0.05121475197315839, -0.1643989873053573, -0.3285206249412727], 1: [0.21693045781865616, 0.0, 0.1699069165076462, 0.43659808637353265, 0.7539903310360807, 0.965201373199428, 0.9976194347048498, 0.9113705996586748, 0.7230304195435754, 0.3967340192080511, 0.24621212076709, 0.0482529058020863, 0.2982749931359468, 0.28, 0.0, 0.1198201601

In [102]:
file = open('BMDS.txt','w') 
file.write('720\n')
c=0
for patient in patient_seg_nodes_dict.keys():
    for imageposno in patient_seg_nodes_dict[patient].keys():
        if len(patient_seg_dict[patient][imageposno][0]) < 100:
            continue
        c+=1
        file.write('36 0\n')
        for node in range(36):
            if node < 12:
                deg = 13
            else:
                deg = 2
            nbrs = str(patient_seg_adj_dict[node])[1:-1].replace(",", "")
            patient_seg_features2_dict[patient][imageposno][0][node]
            patient_seg_features_dict[patient][imageposno][0][node]
            my_formatted_list = [ '%.4f' % (elem/460.0) for elem in patient_seg_features_dict[patient][imageposno][0][node] ]
            eucl_dist_feat_str = str(my_formatted_list)[1:-1].replace(",", "").replace("'", "")
            my_formatted_list = [ '%.4f' % elem for elem in patient_seg_features2_dict[patient][imageposno][0][node] ]
            angu_dist_feat_str = str(my_formatted_list)[1:-1].replace(",", "").replace("'", "")
            file.write(str(node) + ' ' + str(deg) + ' ' + nbrs + ' ' + eucl_dist_feat_str + ' ' + angu_dist_feat_str + '\n')

file.close()
c

720

In [26]:
c=0
for patient in patient_seg_nodes_dict.keys():
    for imageposno in patient_seg_nodes_dict[patient].keys():
        if len(patient_seg_dict[patient][imageposno][0]) < 100:
            print(patient + imageposno)
            c+=1
c

S309213.0.dcm.png.png
S309222.0.dcm.png.png
S309216.0.dcm.png.png
S304293.499969.dcm.png.png
S304295.499969.dcm.png.png
S304285.499969.dcm.png.png
S304297.499969.dcm.png.png
S304281.499969.dcm.png.png
S304291.499969.dcm.png.png
S304289.499969.dcm.png.png
S304287.499969.dcm.png.png
S304301.499969.dcm.png.png
S304283.499969.dcm.png.png
S304299.499969.dcm.png.png


14

In [43]:
str(patient_seg_adj_dict[0])[1:-1].replace(",", "")

'1 2 3 4 5 6 7 8 9 10 11 12 35'

In [52]:
print(patient_seg_features2_dict['S205']['-45.0.dcm.png.png'][0][6])

[-0.9177221168073145, -0.9976194347048498, -0.9419527277358626, -0.780364362336882, -0.6476484200955405, -0.3303504247281061, 0.0, 0.0, -0.07192859021726765, -0.4165299892152057, -0.5052984058745714, -0.681966025158547, -0.9546480106545846, -0.983188736076488, -0.9979253089684582, -0.9796732790158699, -0.8981770081227867, -0.8379980847074857, -0.7435806952781954, -0.7170879554583683, -0.612571665435814, -0.512457509479251, -0.284088329691374, -0.20952908873087345, 0.0, 0.0, 0.0, -0.005681726470806178, -0.1969218046296394, -0.30711355756722536, -0.4514408468529477, -0.47671876051376627, -0.5591416682777818, -0.6219402663483028, -0.7734784404655207, -0.8526012524109426]


In [64]:
my_formatted_list = [ '%.4f' % elem for elem in patient_seg_features2_dict['S205']['-45.0.dcm.png.png'][0][6] ]

In [69]:
str(my_formatted_list)[1:-1].replace(",", "").replace("'", "")

'-0.9177 -0.9976 -0.9420 -0.7804 -0.6476 -0.3304 0.0000 0.0000 -0.0719 -0.4165 -0.5053 -0.6820 -0.9546 -0.9832 -0.9979 -0.9797 -0.8982 -0.8380 -0.7436 -0.7171 -0.6126 -0.5125 -0.2841 -0.2095 0.0000 0.0000 0.0000 -0.0057 -0.1969 -0.3071 -0.4514 -0.4767 -0.5591 -0.6219 -0.7735 -0.8526'

In [87]:
my_formatted_list = [ '%.4f' % (elem/460.0) for elem in patient_seg_features_dict['S205']['-45.0.dcm.png.png'][0][6] ]

In [89]:
str(my_formatted_list)[1:-1].replace(",", "").replace("'", "")

'0.4761 0.4729 0.4662 0.4207 0.2853 0.0921 0.0000 0.2457 0.4533 0.6002 0.6625 0.6599 0.4600 0.4643 0.4727 0.4660 0.4599 0.4462 0.3771 0.3244 0.2200 0.1570 0.0612 0.0311 0.0804 0.1630 0.3130 0.3826 0.5189 0.5734 0.6164 0.6430 0.6765 0.6746 0.6071 0.5533'